In [1]:
import time
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn import svm
import ast
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
#The file_list will contain the files that will be used for training on the Chest X-ray 14 dataset
#The test_list will contain the files that will be used for testing
file_list = []
test_list = []
#Path to the npz files that contain the feature representations
directory =  R'D:\SSD downloads\Processed Chest X-ray 14\\'


for filename in os.listdir(directory):
    #This stores the last 11 npz files for testing, otherwise they are added to the file_list
    for number in range(110, 121):
        if str(number) in filename:
            path = os.path.join(directory, filename)
            if os.path.isfile(path):
                test_list.append(path)
        else:
            path = os.path.join(directory, filename)
            if os.path.isfile(path):
                file_list.append(path)

### Training on Chest X-ray14

In [ ]:
#The classifiers list will contain the 14 different classifers for each different class
classifiers = []
#counter is used to stop the model training
counter = 0

#Loop over the npz files and load them, depending on what key they are stored as, they're loaded into the X_train or y_train 
#lists, once all the values from a npz file have been placed into the list, they are used to train all 14 classifers
for file in file_list:
    #These variables are reset for new file that is loaded
    loaded_array = np.load(file)
    train_image_data = []
    train_label = []
    #Checking what key the current value from the npz file has
    for key in loaded_array:
        if "Label" in key:
            train_label.append(loaded_array[key].tolist())
        else:
            train_image_data.append(loaded_array[key])    
    #Instantiate X_train and y_train
    y_train = train_label
    y_train = np.array(y_train)
    X_train = train_image_data
    #y_train.shape[1] = the number of classes
    #Loop over the classifiers if they exist train them using their index position within the list
    #If they don't exist train them and appende them to the list
    for i in range(y_train.shape[1]):
        if len(classifiers) > i:
            classifiers[i].fit(X_train, y_train[:,i])
        else:
            clf = svm.SVC(probability=True, kernel='rbf', gamma='scale', C=10)
            clf.fit(X_train, y_train[:,i])
            classifiers.append(clf)
    #for every batch of 500 data that is used, counter increases by 1
    counter += 1
    print(counter)
    if counter == 20:
        break

### Testing on Chest X-ray14

In [37]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#counter is used to stop the model testing after x amount of iterations 
counter = 0
#AUC score will be stored in roc_auc variable
roc_auc = 0


for file in test_list:
    #These variables are reset for new file that is loaded
    X_test = []
    y_test = []
    loaded_array = np.load(file)
    #Checking what key the current value from the npz file has
    for key in loaded_array:
        if "Label" in key:
            y_test.append(loaded_array[key].tolist())
        else:
            X_test.append(loaded_array[key])    
    y_test = np.array(y_test)
    #The output of the probability predictions for the different classes will be stored in the y_prob variable
    y_prob = []
    #Generate all the probability predictions from all of the different classifiers
    for i in range(y_test.shape[1]):
        y_prob.append(classifiers[i].predict_proba(X_test)[:,1])
    y_prob = np.array(y_prob).T
    #Calculate the AUC-ROC for each label
    for i in range(y_test.shape[1]):
        #These values represent the indexs of [Effusion, Atelectasis, Consolidation, Cardiomegaly, Edema] in the y_prob[:,i] list
        if i in [2,3,8,7,12]:
            roc_auc += roc_auc_score(y_test[:, i], y_prob[:, i])
            print(f"AUC for label {i}: {roc_auc_score(y_test[:, i], y_prob[:, i])}")
    counter += 1
    if counter == 2:
        #AUC average score is calculated based of how many files have been iterated over
        roc_auc /= 5*counter
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
AUC for label 2: 0.35820363549958417
AUC for label 3: 0.45429553264604805
AUC for label 7: 0.5708841463414634
AUC for label 8: 0.9502032520325204
AUC for label 12: 0.6957831325301205
0
1
2
3
4
5
6
7
8
9
10
11
12
13
AUC for label 2: 0.4567099567099567
AUC for label 3: 0.6336734693877552
AUC for label 7: 0.6055327868852458
AUC for label 8: 0.9805327868852459
AUC for label 12: 0.932260228034876


In [38]:
#Average AUC score for in-distribution data
print(roc_auc)

0.6638078926952816


### Testing on CheXpert

In [39]:
#All the CheXpert files will be loaded into the test_list for out of distribution evaluation
test_list = []
directory =  R'Q:\Processed CheXpert'

for filename in os.listdir(directory):
    path = os.path.join(directory, filename)
    if os.path.isfile(path):
        test_list.append(path)
    #After 5 files have been added to test_list it breaks the loop
    if len(test_list) == 5:
        break

In [41]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#counter is used to stop the model testing after x amount of iterations 
counter = 0
#AUC score will be stored in roc_auc variable
roc_auc = 0


for file in test_list:
    #These variables are reset for new file that is loaded
    X_test = []
    y_test = []
    loaded_array = np.load(file)
    #Checking what key the current value from the npz file has
    for key in loaded_array:
        if "Label" in key:
            y_test.append(loaded_array[key].tolist())
        else:
            X_test.append(loaded_array[key])    
    y_test = np.array(y_test)
    #The output of the probability predictions for the different classes will be stored in the y_prob variable
    y_prob = []
    #Generate all the probability predictions from all of the different classifiers
    for i in range(y_test.shape[1]):
        y_prob.append(classifiers[i].predict_proba(X_test)[:,1])
    y_prob = np.array(y_prob).T
    #Calculate the AUC-ROC for each label
    for i in range(y_test.shape[1]):
        #These values represent the indexs of [Edema, Effusion, Atelectasis, Consolidation, Cardiomegaly] in the y_prob[:,i] list
        if i in [0,6,7,9,10]:
            roc_auc += roc_auc_score(y_test[:, i], y_prob[:, i])
            print(f"AUC for label {i}: {roc_auc_score(y_test[:, i], y_prob[:, i])}")
    counter += 1
    if counter == 2:
        #AUC score is calculated based of how many files have been iterated over
        roc_auc /= 5*counter
        break

AUC for label 0: 0.6325639426076107
AUC for label 6: 0.4668850072780204
AUC for label 7: 0.4288842784693407
AUC for label 9: 0.5116120218579235
AUC for label 10: 0.5450668510834487
AUC for label 0: 0.5415367020170513
AUC for label 6: 0.5839571636514869
AUC for label 7: 0.5586675887505763
AUC for label 9: 0.5166078777189889
AUC for label 10: 0.48245352420717763


In [42]:
#Average AUC score for out-of-distribution data
print(roc_auc)

0.5268234957641625